## 3 层神经网络的实现

通过前面几节的基础知识的介绍，我们对神经网络的组成有了一定的认识，我们这里就将演示一个完整的3层神经网络的实际演化的过程层。同时我们也将使用上一小节中，Numpy对于矩阵操作的技巧。

先看看本节需要实现的神经网络的样子：

![3层神经网络](imgs/10.jpg)

- 输入为[x1,x2]
- 第一层3个神经元
- 第二层2个神经元
- 输出层y1,y2

### 表示符号的约定

由于神经网络中符号既要表示出符号处在哪一层，又要表示在该层中的位置，所以这里我们的符号，稍微有点复杂，但是也不是很难。

- 输出层的符号：$x1,x2$ 里面的[1,2,3...]表示输入的数组的序列号。
- 权重$W_{12}^{(1)}$ 
    - (1): 表示第1层的权重，第几层的权重
    - 12： 该权重连接的后一层和前一层的第几个神经元
        - 1：表示后一层的第1一个神经元
        - 2：表示前一层的第2一个神经元
        
![](imgs/11.jpg)

- 隐藏层$a_1^{(1)}$
    - 下标表示：在该层中的序列位置
    - 上标表示：在第几层的位置
    
    
### 信号的传递的过程

#### 输入层传输到第一层信号量过程

我们来看一下从输入层到第一层的某一神经元的信号传递过程。

![信号传递](imgs/12.jpg)

上图显示信号传递的过程，里面在输入层添加了一个1，用来进行偏置的数据输入。这里主要是为了方便将整个过程讲清楚而添加的，实际操作中，我们会直接加上b。

其实所谓的信号传递，就是通过我们以前介绍的公式来实现，没有那么复杂。比如我们要计算，传递给该神经元的信号量为多少,具体公式如下：  
$a_1^{(1)} = w_11^{(1)}x_1 + w_12^{(1)}x_2 + b_1^{(1)}  $

上面第一层具体一个神经元的计算公式，我们如果使用矩阵的形式，就可以将整个的该层计算出来。  
$A^{(1)}=XW^{(1)}+B^{(1)}$

       
  

In [3]:
# 从输入层到第一层的信号传递过程
import numpy as np

# X 输入层，一维数组
X = np.array([1,2])
# W 连接输入层2，第一层神经元个数为3，所以构成（2，3）的矩阵
W = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])
# B 是一个一维的数组，size是第一层神经元个数为3
B = np.array([0.1,0.2,0.3])

print(X.shape)
print(W.shape)
print(B.shape)

# 信号传递到具体的神经元计算如下
A1 = np.dot(X,W)+B

print(A1)
print(A1.shape)


(2,)
(2, 3)
(3,)
[0.6 1.3 2. ]
(3,)


#### 第一层激活的过程

下面信号量已经传递过来了，我们需要利用激活来激活这个神经元。具体如下图：

![](imgs/13.jpg)


这里激活函数我们选用sigmoid函数。上图中的h()就是使用sigmoid函数。


In [4]:
def sigmoid(x):
    return 1.0/(np.exp(-x)+1)

In [5]:
Z1 = sigmoid(A1)
print(Z1)

[0.64565631 0.78583498 0.88079708]


#### 第一层到第二层信号传递过程

这里我们就可以参考上面的过程，使用的计算公式都是一样的：  
$A^{(2)}=Z1W^{(2)}+B^{(2)}$  
$Z2 = sigmoid(A^{(2)})$  
具体的过程如下图：
![](imgs/14.jpg)

In [8]:
# 第一层3个神经元，第二层2个神经元所以W2(3,2)
W2 = np.array([[0.1,0.2],[0.3,0.4],[0.5,0.6]])
# 第二层2个神经元，B2(2,)
B2 = np.array([0.1,0.2])
print(W2.shape)
print(B2.shape)
A2 = np.dot(Z1,W2) + B2
print(A2)
print(A2.shape)

Z2 = sigmoid(A2)
print(Z2)

(3, 2)
(2,)
[0.84071466 1.1719435 ]
(2,)
[0.69861571 0.76349613]


#### 第二层到输出层信号传递过程

这里我们要注意，关于输出层激活函数的选择，要根据实际的问题来选用适当的激活函数，比如：
- 二分类问题可以选用：sigmoid
- 多分类可以选用：softmax
- 回归则可以选用这里的恒等，其实就是输出一个特定的值。

这里为了统一就定义一个简单的恒等函数，其实没有什么意义。
具体的过程如下图：
![](imgs/15.jpg)


In [9]:
def identity_fun(x):
    return x

In [10]:
# 第二层2个神经元，输出层2个神经元所以W2(3,2)
W3 = np.array([[0.1,0.2],[0.3,0.4]])
B3 = np.array([0.1,0.2])
print(W3.shape)
A3 = np.dot(Z2,W3)+B3
print(A3.shape)
output = identity_fun(A3)
print(output)

(2, 2)
(2,)
[0.39891041 0.6451216 ]


最好将上面的代码整合到一起，具体如下：
先定义号网络的需要的参数，其实这个就是在定义网络的层数和神经元的个数。
再定义一个信号传递的函数，信号前向传播函数。具体如下code.


In [17]:
def init_network():
    net={}
    net['W1'] = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])
    net['W2'] = np.array([[0.1,0.2],[0.3,0.4],[0.5,0.6]]) 
    net['W3'] = np.array([[0.1,0.2],[0.3,0.4]])
    net['B1'] = np.array([0.1,0.2,0.3])
    net['B2'] = np.array([0.1,0.2])
    net['B3'] = np.array([0.1,0.2])
    
    return net

In [22]:
# 前向传播网络
def forward(network,x):
    W1,W2,W3 = network['W1'],network['W2'],network['W3']
    b1,b2,b3 = network['B1'],network['B2'],network['B3']
    
    # 第一层
    A1 = np.dot(x,W1)+b1
    Z1 = sigmoid(A1)
    #第二层
    A2 = np.dot(Z1,W2)+b2
    Z2 = sigmoid(A2)
    #输出层
    Z3 = np.dot(Z2,W3) + b3
    y = identity_fun(Z3)
    return y
    

In [23]:
# 构建网络
network = init_network()
x = np.array([0.5,0.6])

y = forward(network,x)
print(y)


[0.3861894  0.62622355]
